In [1]:
%reload_ext autoreload
%autoreload 2

In [19]:
import os,sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from scipy.spatial.distance import euclidean
import numpy as np

os.chdir('..')

In [20]:
from src.data_preprocess import DataPreProcessor
from sqlalchemy import create_engine
from src.config import DBConfig
db_config = DBConfig.load()
preprocessor = DataPreProcessor()

df_raw = preprocessor.fetch_preprocessed_data()
msisdn_to_exclude = 41882819545.02736
df = df_raw[df_raw['MSISDN/Number'] != msisdn_to_exclude]

In [22]:
engagement_metrics = df.groupby('MSISDN/Number').agg({'Bearer Id': 'count', 
                                                       'Dur. (ms)': 'sum', 
                                                       'Total DL (Bytes)': 'sum', 
                                                       'Total UL (Bytes)': 'sum'})
engagement_metrics.columns = ['Sessions', 'Duration', 'Total_DL', 'Total_UL']  # Rename columns

# Task 4: Aggregate experience metrics per customer
experience_metrics = df.groupby('MSISDN/Number').agg({'TCP DL Retrans. Vol (Bytes)': 'mean',
                                                       'Avg RTT DL (ms)': 'mean',
                                                       'Avg Bearer TP DL (kbps)': 'mean'})


In [23]:
scaler_engagement = StandardScaler()
engagement_normalized = scaler_engagement.fit_transform(engagement_metrics)

# Task 4: Normalize experience metrics
scaler_experience = StandardScaler()
experience_normalized = scaler_experience.fit_transform(experience_metrics)

# Task 3: K-means clustering for engagement
kmeans_engagement = KMeans(n_clusters=3, random_state=42)
kmeans_engagement.fit(engagement_normalized)

# Task 4: K-means clustering for experience
kmeans_experience = KMeans(n_clusters=3, random_state=42)
kmeans_experience.fit(experience_normalized)

# Task 5.1: Calculate engagement score for each user
engagement_cluster_centroids = kmeans_engagement.cluster_centers_
engagement_scores = np.sqrt(((engagement_normalized - engagement_cluster_centroids[0])**2).sum(axis=1))

# Task 5.1: Calculate experience score for each user
experience_cluster_centroids = kmeans_experience.cluster_centers_
experience_scores = np.sqrt(((experience_normalized - experience_cluster_centroids[2])**2).sum(axis=1))

# Task 5.2: Calculate satisfaction score
satisfaction_scores = (engagement_scores + experience_scores) / 2

satisfaction_df = pd.DataFrame({'MSISDN/Number': engagement_metrics.index, 'Satisfaction Score': satisfaction_scores})

# Task 5.2: Top 10 satisfied customers
top_10_satisfied_customers = satisfaction_df.nlargest(10, 'Satisfaction Score')

# Output top 10 satisfied customers
print("Top 10 Satisfied Customers:")
print(top_10_satisfied_customers)

Top 10 Satisfied Customers:
       MSISDN/Number  Satisfaction Score
47983   3.366232e+10           85.657430
13180   3.362578e+10           62.760557
80249   3.368369e+10           49.756373
6437    3.361489e+10           41.383235
92923   3.376054e+10           39.355135
13526   3.362632e+10           38.581799
65118   3.366716e+10           36.576656
41944   3.366087e+10           30.227219
666     3.360313e+10           29.892219
50281   3.366284e+10           29.716932


In [24]:


connection_string = f"postgresql+psycopg2://{db_config['DATABASE_USER']}:{db_config['DATABASE_PASSWORD']}@{db_config['DATABASE_HOST']}:{db_config['DATABASE_PORT']}/{db_config['DATABASE_NAME']}"
engine = create_engine(connection_string)

# Export satisfaction_df to MySQL database
satisfaction_df.to_sql(name='user_satisfaction_scores', con=engine, if_exists='replace', index=False)

# Example query 
query = "SELECT * FROM user_satisfaction_scores LIMIT 50"
select_output = pd.read_sql(query, con=engine)
print(select_output)


    MSISDN/Number  Satisfaction Score
0    3.360100e+10           12.456515
1    3.360100e+10           12.775478
2    3.360100e+10           12.503300
3    3.360101e+10           12.909930
4    3.360101e+10           11.974209
5    3.360101e+10           12.058793
6    3.360101e+10           11.926906
7    3.360101e+10           12.782178
8    3.360101e+10           12.156905
9    3.360102e+10           12.391475
10   3.360102e+10           12.811728
11   3.360102e+10           12.472789
12   3.360102e+10           12.472291
13   3.360102e+10           12.542419
14   3.360103e+10           12.667500
15   3.360103e+10           12.834442
16   3.360103e+10           12.522565
17   3.360103e+10           12.543295
18   3.360103e+10           12.612821
19   3.360103e+10           12.467688
20   3.360103e+10           12.138632
21   3.360103e+10           11.558023
22   3.360104e+10           12.624685
23   3.360104e+10           12.868468
24   3.360104e+10           12.707828
25   3.36010